# Tutorial-1

This tutorial explains the steps to be followed to generate the series of compressor
efficiency ('eta_s') values for the design point data of a particular heat pump model.
In this example, the data for Daikin's Air/Water Heat Pump (Model  ERLQ016CAV3) has been
used.


### Parametrizing the model for the nominal capacity data

Following the advice given in the documentation, the different components of the network
have to be parametrized such that the compressor power calculated by the model matches that
of the data sheet, for the nominal capacity data. For the heat pump model in this example, the
heat load is 16.3 kW & the power required is 3.83 kW, at an ambient air temperature of 7 deg. C
and heat supply temperature of 35 deg. C.

Code: 'Parametrization_NominalData.py'

The network solution for this case should be saved as it will be used in the following steps.
The network is saved as 'Par_NominalData'.


### Setting up the input data files

The data from the heating capacity tables should be available in csv format.
The heating capacities (in kW) and power requirement (in kW) at the different evaporator
and condenser temperatures should be available as shown in the 'Data\Heat_Load_Data.csv' and
'Data\PI_Data.csv' respectively. The first row contains the condenser leaving temperatures
and the first column contains the ambient temperature. The temperatures are in deg. C.


In [1]:
from tespy.networks import load_network
import pandas as pd
import numpy as np
from tqdm import tqdm
import json

In [3]:
# import the saved network with the initial parametrization data

imported_plant = load_network('.\Par_NominalData')
# imported_plant.solve('design')
# imported_plant.print_results()

In [4]:
#import the heat load data

HL_Data_Path = '.\Data\Heat_Load_Data.csv'
heat_load_data = pd.read_csv(HL_Data_Path, header=0, delimiter=',')
print(heat_load_data)

   T_amb     30     35     40     45     50     55
0    -20   9.56   9.66   9.59   7.69    NaN    NaN
1    -15  10.57  10.56   9.86   9.55   8.79    NaN
2     -7  12.59  12.30  12.02  11.35  10.26   9.18
3     -2  12.11  11.79  11.48  11.39  10.44   9.32
4      2  11.74  11.40  11.07  11.37  11.04  10.29
5      7  16.63  16.10  15.47  15.22  14.51  13.92
6     12  17.34  16.74  16.13  15.76  15.13  14.51
7     15  18.81  18.16  17.51  17.10  16.43  15.75
8     20  21.49  20.77  20.04  19.59  18.83  18.07


In [5]:
#import the power requirement data

PI_Data_Path = '.\Data\PI_Data.csv'
PI_data = pd.read_csv(PI_Data_Path, header=0, delimiter=',')
print(PI_data)


   T_amb    30    35    40    45    50    55
0    -20  5.67  6.07  6.40  6.38  0.00  0.00
1    -15  5.84  6.28  6.30  6.34  6.38  0.00
2     -7  5.07  5.49  5.95  6.34  6.37  6.37
3     -2  4.32  4.71  5.15  5.63  5.86  5.86
4      2  3.75  4.09  4.48  4.84  5.51  5.88
5      7  3.55  3.83  4.26  4.71  5.17  5.71
6     12  3.45  3.78  4.15  4.58  5.05  5.58
7     15  3.45  3.78  4.16  4.58  5.06  5.59
8     20  3.43  3.77  4.15  4.59  5.07  5.61


In [6]:
# A DataFrame to save the power calculated by the model

PI_Data_new= pd.DataFrame().reindex_like(PI_data)
PI_Data_new['T_amb'] = PI_data['T_amb']
print( PI_Data_new)

   T_amb  30  35  40  45  50  55
0    -20 NaN NaN NaN NaN NaN NaN
1    -15 NaN NaN NaN NaN NaN NaN
2     -7 NaN NaN NaN NaN NaN NaN
3     -2 NaN NaN NaN NaN NaN NaN
4      2 NaN NaN NaN NaN NaN NaN
5      7 NaN NaN NaN NaN NaN NaN
6     12 NaN NaN NaN NaN NaN NaN
7     15 NaN NaN NaN NaN NaN NaN
8     20 NaN NaN NaN NaN NaN NaN


In [7]:
# The Leaving Water Condenser (LWC) temperatures
cols = heat_load_data.columns[1:]
print(cols)

Index(['30', '35', '40', '45', '50', '55'], dtype='object')


In [8]:
# The range of values for the efficiency of the compressor from which the best
# value is chosen
etas_list = np.arange(0.2,1.3,0.01)

In [9]:
# A DataFrame to save the best efficiency values identified for each design point
etas_data = pd.DataFrame().reindex_like(heat_load_data)
etas_data['T_amb'] = heat_load_data['T_amb']
print(etas_data)

   T_amb  30  35  40  45  50  55
0    -20 NaN NaN NaN NaN NaN NaN
1    -15 NaN NaN NaN NaN NaN NaN
2     -7 NaN NaN NaN NaN NaN NaN
3     -2 NaN NaN NaN NaN NaN NaN
4      2 NaN NaN NaN NaN NaN NaN
5      7 NaN NaN NaN NaN NaN NaN
6     12 NaN NaN NaN NaN NaN NaN
7     15 NaN NaN NaN NaN NaN NaN
8     20 NaN NaN NaN NaN NaN NaN


In [10]:
total = len(etas_data.index) * len(cols)

In [11]:
pbar = tqdm(total=total)
for i in range(9):
    T_amb = heat_load_data['T_amb'][i] # The ambient temperature temperature
    k = 30 # index for moving within etas_list
    for j in cols:
        LWC = float(j) # The Leaving Water Condenser (LWC) temperature
        HL = float(heat_load_data[j][i]*1000) # Heat Load in W

        l=1
        if  not np.isnan(HL):
            # print ('row: %d , column: %s'%(i,j))
            # The display of iteration info during network solving is turned off
            imported_plant.set_attr(iterinfo=False)

            # A temperature difference of 5 deg.C is assumed across the condenser
            imported_plant.get_conn('condenser:out2_consumer:in1').T.val= LWC
            imported_plant.get_conn('consumer cycle closer:out1_condenser recirculation pump:in1').T.val =LWC-5

            # A temperature difference of 5 deg.C is assumed across the evaporator
            imported_plant.get_conn('source ambient:out1_ambient pump:in1').T.val=T_amb
            imported_plant.get_conn('evaporator:out1_sink ambient:in1').T.val= T_amb - 5

            # The heat load is set
            imported_plant.get_comp('consumer').set_attr(Q=-HL)

            # print("row: %d, column: %s" %(i,j))

            # The efficiency value for the compressor that gives the power closest
            # to the power from the data sheet is found
            while 0 <= k < len(etas_list):

                imported_plant.get_comp('compressor').eta_s.val = etas_list[k]
                imported_plant.solve('design')

                P_new = imported_plant.get_comp('compressor').P.val/1000

                P_new_dif = P_new - PI_data.loc[i, j]

                if abs(P_new_dif) > 0.05:

                    if P_new_dif < 0:
                        k -= 1
                    else:
                        k += 1

                elif abs(P_new_dif) <= 0.05 and P_new_dif < 0:
                    # print("entering 2nd condition: %d time" %l)
                    k -= 1
                    P_old = P_new
                    P_old_dif = P_new_dif
                    # print(P_old, P_old_dif)
                    imported_plant.get_comp('compressor').eta_s.val = etas_list[k]
                    imported_plant.solve('design')
                    P_new = imported_plant.get_comp('compressor').P.val/1000
                    P_new_dif = P_new - PI_data.loc[i, j]
                    # print(P_new, P_new_dif)
                    if round(abs(P_new_dif),2) <= round(abs(P_old_dif),2):
                        # print("entering 2-1 condition")
                        PI_Data_new.loc[i,j] = P_new
                        etas_data.loc[i,j] = etas_list[k]
                    else:
                        # print("entering 2-2 condition")
                        k += 1
                        PI_Data_new.loc[i,j] = P_old
                        etas_data.loc[i,j] = etas_list[k]
                    # print(PI_Data_new[j][i], etas_data[j][i])
                    break

                elif abs(P_new_dif)<=0.05 and P_new_dif > 0:
                    # print("entering 3rd condition: %d time" %l)
                    k += 1
                    P_old = P_new
                    P_old_dif = P_new_dif
                    imported_plant.get_comp('compressor').eta_s.val = etas_list[k]
                    imported_plant.solve('design')
                    P_new = imported_plant.get_comp('compressor').P.val/1000
                    P_new_dif = P_new - PI_data.loc[i,j]
                    if round(abs(P_new_dif),2) < round(abs(P_old_dif),2):
                        PI_Data_new.loc[i,j] = P_new
                        etas_data.loc[i,j] = etas_list[k]
                    else:
                        k -= 1
                        PI_Data_new.loc[i,j] = P_old
                        etas_data.loc[i,j] = etas_list[k]
                    # print(PI_Data_new[j][i], etas_data[j][i])
                    break

        else:
            etas_data.loc[i,j] = np.nan

        pbar.update()

pbar.close()

100%|██████████| 54/54 [01:15<00:00,  1.40s/it]


In [11]:
print(etas_data)


   T_amb    30    35    40    45    50    55
0    -20  0.32  0.31   NaN   NaN   NaN   NaN
1    -15  0.33  0.32  0.31  0.32   NaN   NaN
2     -7  0.50  0.48  0.46  0.42  0.39  0.38
3     -2  0.53  0.52  0.49  0.48  0.45  0.45
4      2  0.56  0.55  0.53  0.58  0.52  0.52
5      7  0.84  0.86  0.83  0.84  0.83  0.88
6     12  0.78  0.80  0.80  0.81  0.81  0.87
7     15  0.78  0.80  0.81  0.83  0.84  0.91
8     20  0.77  0.81  0.83  0.86  0.89  0.98


In [14]:
etas_data.to_csv('parametrization_etas_R1.csv')

In [12]:
print(PI_Data_new - PI_data)

   T_amb        30        35        40        45        50        55
0      0 -0.008062  0.027468       NaN       NaN       NaN       NaN
1      0  0.033205  0.017013 -0.013485  0.020716       NaN       NaN
2      0  0.007375  0.023334  0.010212  0.000161  0.013341 -0.010126
3      0  0.019364 -0.019039  0.013686  0.018791 -0.017835  0.000170
4      0  0.018680  0.012508  0.013659 -0.009072  0.009972 -0.000793
5      0  0.005435  0.008335  0.021189  0.012072  0.006553  0.012143
6      0  0.015181 -0.013504 -0.007199 -0.005508 -0.000886 -0.006266
7      0  0.010195  0.016210  0.011916  0.016367  0.012085  0.015542
8      0 -0.003599 -0.014493  0.005732  0.014563 -0.005013  0.000105


In [ ]:
#Convert it to json? or some other type?

In [12]:
etas_data.set_index('T_amb', inplace=True)
result = etas_data.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)

'{\n    "-20": {\n        "30": 0.32,\n        "35": 0.31,\n        "40": null,\n        "45": null,\n        "50": null,\n        "55": null\n    },\n    "-15": {\n        "30": 0.33,\n        "35": 0.32,\n        "40": 0.31,\n        "45": 0.32,\n        "50": null,\n        "55": null\n    },\n    "-7": {\n        "30": 0.5,\n        "35": 0.48,\n        "40": 0.46,\n        "45": 0.42,\n        "50": 0.39,\n        "55": 0.38\n    },\n    "-2": {\n        "30": 0.53,\n        "35": 0.52,\n        "40": 0.49,\n        "45": 0.48,\n        "50": 0.45,\n        "55": 0.45\n    },\n    "2": {\n        "30": 0.56,\n        "35": 0.55,\n        "40": 0.53,\n        "45": 0.58,\n        "50": 0.52,\n        "55": 0.52\n    },\n    "7": {\n        "30": 0.84,\n        "35": 0.86,\n        "40": 0.83,\n        "45": 0.84,\n        "50": 0.83,\n        "55": 0.88\n    },\n    "12": {\n        "30": 0.78,\n        "35": 0.8,\n        "40": 0.8,\n        "45": 0.81,\n        "50": 0.81,\n     

In [13]:
data={}
data['eta_s'] = parsed

In [14]:
heat_load_data.set_index('T_amb', inplace=True)
result = heat_load_data.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)
data['heatload'] = parsed

In [15]:
ids = np.arange(1,etas_data.size+1,1)
ids_x = [ids[x:x+len(etas_data.columns)] for x in range(0,len(ids), len(etas_data.columns)) ]
id_list = pd.DataFrame(ids_x, columns=etas_data.columns, index=etas_data.index)
result = id_list.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)
data['ids'] = parsed

In [20]:
# The minimum heatload for this model of heat pump should be provided
data['min_heatload'] = 5000
data['max_heatload'] = 1000* heat_load_data.max().max()

In [3]:
# Give a name to the model of heat pump being parametrized
model_name = 'Air_16kW'
with open("../../mosaik_heatpump/heatpump/eta_s_data.json") as feedsjson:
    feeds = json.load(feedsjson)

feeds['Air_8kW_mod'] = feeds['Air_8kW']

if model_name in feeds.keys():
    a = input('The model name already exists and must be replaced. Do you want to replace the data (y/n)? : ')
    if a.lower() == 'y':
        feeds[model_name]= data
    elif a.lower() =='n':
        model_name = input('Enter new model name')
        feeds[model_name]= data
    else:
        print("Wrong input. Execute cell again.")
else:
    feeds[model_name] = data

In [4]:
with open("../../mosaik_heatpump/heatpump/eta_s_data.json", "w") as write_file:
    json.dump(feeds, write_file, indent=4)